# XKCD  - Should you automate it?

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://imgs.xkcd.com/comics/is_it_worth_the_time.png")


This table gets posted a lot in programming forums, in order to poke fun at the idea that programmers will spend 5 hours failing to automate something rather than doing it manually in 5 minutes. 

This chart claims to guide you on figuring out if a task is worth automating or not.

However the formula behind the chart is not obvious, so I figured, I should probably try to figure that out.



In [2]:
#Checking cell[0][0]

frequency = 50
time_you_spend = 24*60*60
timeframe = 5*365
saved_unit = 1

time_you_saved = frequency*saved_unit*timeframe


print(time_you_saved/time_you_spend)

1.0561342592592593


It seems like if time saved is 5% more than time spent, then its worth it to the creator of the image, need to try with another cell.

In [3]:
#Checking cell[0][1]

frequency = 5
time_you_spend = 2*60*60
timeframe = 5*365
saved_unit = 1

time_you_saved = frequency*saved_unit*timeframe

print(time_you_saved/time_you_spend)

1.2673611111111112


26% more this time, hmmmm, need to try with another cell.

In [4]:
#Checking cell[3][4]

frequency = 1/7
time_you_spend = 21*60*60
timeframe = 5*365
saved_unit = 5*60

time_you_saved = frequency*saved_unit*timeframe


print(time_you_saved/time_you_spend)

1.034580498866213


3% more this time. So its safe to confirm he doesn't have a solid criteria, but more of a hunch, which I should have realized instantly given all the cells are clean numbers and not floats like 3.0583 days; That over 5 years if the ratio of time spent automating to time saved is in the favor of time saved then its all good. Maybe a more general function would be better. 

Hasty conclusion? Maybe, but I would rather spend 2 minutes testing 3 random cells than turning his table into a dataframe and reaching the same conclusion with 99% confidence after 2 hours. 

In [44]:
import typing
import re

def should_you_automate_it(frequency: str,time_saved: str,time_range: str):
    # Yes 90% of the function is user input string formatting.
    """
    Args:
    frequency = How many times per unit of time do you plan to do the task
        format = "1 / hour", float seperated by "/" then unit in str (from seconds to years)
        white space doesn't matter
    
    time_saved = How much time would automation save every time you do said thing
        format = "1  hour", float seperated by unit in str (from seconds to years)
        
    time_range = Over how long will you use said automation
        format = "1  hour", float seperated by unit in str (from seconds to years)
        
        
        Formatting is such that it's easier to take user input in whatever units they want
        
    """
    
    def preprocess_time(s: str):
        """
        splits a number followed by a string and returns a tuple
        
        """
        pattern = re.compile(r'\s+')
        s = re.sub(pattern, '', s)
        match = re.match(r"([0-9]+)([a-z]+)", s, re.I)
        if match:
            return match.groups()
        
    def preprocess_time_relative(s: str):
        
        """
        splits into parts like previous function
        """
    
        return re.findall(r"[\w']+|[/]",s)
    
    def secondsToText(secs: float):
        days = secs//86400
        hours = (secs - days*86400)//3600
        minutes = (secs - days*86400 - hours*3600)//60
        seconds = secs - days*86400 - hours*3600 - minutes*60
        result = ("{0} day{1}, ".format(days, "s" if days!=1 else "") if days else "") + \
        ("{0} hour{1}, ".format(hours, "s" if hours!=1 else "") if hours else "") + \
        ("{0} minute{1}, ".format(minutes, "s" if minutes!=1 else "") if minutes else "") + \
        ("{0} second{1}, ".format(seconds, "s" if seconds!=1 else "") if seconds else "")
        return result
    
    
    plural_dict = {'second': 1, 'minute': 60, 'hour': 60*60 
                   ,'day': 60*60*24, 'week': 60*60*24*7, 'month': 60*60*24*30
                   , 'year' : 60*60*24*365 } #Convert all to seconds to make my job easier.
    
    frequency_number, frequency_unit = float(preprocess_time_relative(frequency)[0]), preprocess_time_relative(frequency)[2]
    time_saved_number, time_saved_unit = float(preprocess_time(time_saved)[0]),preprocess_time(time_saved)[1]
    time_range_number, time_range_unit = float(preprocess_time(time_range)[0]),preprocess_time(time_range)[1]
    
    plural_dict_multiplier_freq = plural_dict[frequency_unit] 
    plural_dict_multiplier_saved = plural_dict[time_saved_unit[0:-1]] if time_saved_number != 1 else plural_dict[time_saved_unit]
    plural_dict_multiplier_range = plural_dict[time_range_unit[0:-1]] if time_range_number != 1 else plural_dict[time_range_unit]
    
    frequency_cal = frequency_number/plural_dict[frequency_unit]
    saved_cal = (time_saved_number*plural_dict_multiplier_saved)
    range_cal = (time_range_number*plural_dict_multiplier_range)
    
    total_time_saved = frequency_cal*saved_cal*range_cal
    
    
    #Sanity check
    
    total_time_needed = frequency_number*plural_dict_multiplier_saved
    
    if total_time_needed > plural_dict_multiplier_freq:
        print("Frequency times time saved is more than total per unit check your inputs")
    else:
        print(f"Spend no more than {secondsToText(total_time_saved)}to automate the task.")
        return total_time_saved
    

In [45]:
should_you_automate_it("50 / day","1 hour","5 years")

Frequency times time saved is more than total per unit check your inputs


In [46]:
should_you_automate_it("50 / day","1 minute","5 years")

Spend no more than 63.0 days, 8.0 hours, 50.0 minutes, to automate the task.


5475000.0

In [47]:
should_you_automate_it("100 / hour","2 seconds","3 months")

Spend no more than 5.0 days, to automate the task.


432000.0